<center><b><font size="+3"><font color="blue">Tweetory classify tweets using Watson NLP</font></font></b></center>
<center>Created by Alexandre Rodrigues for Polytech Paris students</center>
<center><a href="https://www.linkedin.com/in/rodriguesa/" target="_blank">Find me on LinkedIn</a></center>

In [202]:
#Set up of watson client with requests
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
import Features, EmotionOptions, CategoriesOptions

#create yourself a Watson Natural Language Understanding service first to get credentials 
watsonUnderstanding = NaturalLanguageUnderstandingV1(
  username='',
  password='',
  version='2017-02-27')

#requests.get('', auth=('user', 'pass'))

In [203]:
#set up of MongoDB client
from pymongo import MongoClient

#mongoDB host / port / database
mongoHost = MongoClient('mongodb://192.168.1.67:27017')

In [204]:
#send tweet text with some cleanup to Watson and store answers to MongoDB

#lib used to manage interupt codes and runtime
import sys
import time

#Get twittos collections names and iterate
for twittoCollectionName in mongoHost.twitterDB.collection_names():
    #build proper collection name for mongoClient
    collection = getattr(mongoHost.twitterDB,twittoCollectionName)
    print("Scaning unclassified tweets of", twittoCollectionName)
    
    #retrieve unclassified tweets, remove langage unknown
    tweets = collection.find({'$and': [{"watsonNLP": None},
                                      {'$or':[{"lang" : "en"},
                                               {"lang" : "fr"},
                                               {"lang" : "pt"}]}]})
    print (tweets.count(), "tweets to classify")
    
    #iterate
    for tweet in tweets:
        #remove unwanted behaviors because of reply to @username
        words = tweet["text"].split()
        for word in words:
            if "@" in word:
                words.remove(word)
        cleanTweet = ' '.join(words)
        
        #prepare collections for watson stats
        watsonCollection = getattr(mongoHost.watsonDB,twittoCollectionName)
        
        #request Watson Natural Langage Processing
        try:
            response = watsonUnderstanding.analyze(
                            text=cleanTweet,
                            features=Features(
                                categories=CategoriesOptions(),
                            emotion=EmotionOptions()),
                            language=tweet["lang"])
            
            #create document key same as tweetID
            insert = watsonCollection.insert_one({"_id": tweet["_id"]})
            update = insertWatson = watsonCollection.update_one(
                {"_id": tweet["_id"]},
                {'$set': response})
            
            #update tweet status on database to avoid futur duplicate work
            update = collection.update_one(
                {"_id": tweet["_id"]},
                {'$set' : {"watsonNLP": "Yes"} })
            sys.stdout.write('.')
        except:
            print("error at line number:",sys.exc_info()[-1].tb_lineno)
            print("message:", sys.exc_info())
            print("Tweetory was processing a tweet from:",tweet["author"], 
                    "with TweetID", tweet["_id"])
    print()
print("Done")

Scaning unclassified tweets of EmmanuelMacron
0 tweets to classify

Scaning unclassified tweets of satyanadella
0 tweets to classify

Scaning unclassified tweets of VasquesXavier
0 tweets to classify

Scaning unclassified tweets of Jae0Pierre
0 tweets to classify

Scaning unclassified tweets of Bouletcorp
5 tweets to classify
.....
Scaning unclassified tweets of David_Cameron
0 tweets to classify

Scaning unclassified tweets of nsekkaki
0 tweets to classify

Scaning unclassified tweets of paulocoelho
1 tweets to classify
.
Scaning unclassified tweets of IBMWatson
0 tweets to classify

Scaning unclassified tweets of BillGates
0 tweets to classify

Scaning unclassified tweets of cathcoste
0 tweets to classify

Scaning unclassified tweets of realDonaldTrump
0 tweets to classify

Scaning unclassified tweets of tim_cook
0 tweets to classify

Scaning unclassified tweets of theresa_may
0 tweets to classify

Scaning unclassified tweets of BarackObama
0 tweets to classify

Scaning unclassified 

In [205]:
#Collect categories from watsonDB and rank them for user
#Set wanted minimal confidence
ConfidenceValue = 0.5
print("The seek confidence value is set to:",ConfidenceValue)

#retrieve tweets categories when confidence rech precision value
for watsonCollectionName in mongoHost.watsonDB.collection_names():
    
    print("Working on watson categories of twitter account:", watsonCollectionName)
    #build collection name adress
    watsonCollection = getattr(mongoHost.watsonDB,watsonCollectionName)
    CategoriesCollection = getattr(mongoHost.categories,watsonCollectionName)
    
    #clean target collection for re-run purposes
    CategoriesCollection.delete_many({})
    counterItems = 0
    counterCategories = 0

    for watsonCategoriesAnswer in watsonCollection.find({'categories.score' :  {"$gt": ConfidenceValue}},
                                                      {'categories' : 1, '_id': 0}):
        
        #Prepare data because it contains list , tuples and string 
        #Needs improvement here. 
        #Likely by changing the way data is inserted (TweETL process)
        #remove 'categories': which doesn't have value added anymore
        ListofwatsonCategories = list(watsonCategoriesAnswer.values())
        WatsonCategories = ListofwatsonCategories[0]
        
        #insert the categories that reach minimum confidence
        for WatsonCategory in WatsonCategories :
            if WatsonCategory['score'] > ConfidenceValue:
                
                counterItems +=1
                
                WatsonCategoryLabel = WatsonCategory['label']
                if CategoriesCollection.find({'label' : WatsonCategoryLabel}).count() > 0:
                    CategoriesCollection.update_one({'label': WatsonCategoryLabel},{"$inc": {'count': 1}})
                else:
                    CategoriesCollection.insert_one({'label': WatsonCategory['label'], 'count': 1})
                    counterCategories += 1
    print ("The program found:",counterItems,"records split into",counterCategories,"categories")
print("Done")

The seek confidence value is set to: 0.5
Working on watson categories of twitter account: paulocoelho
The program found: 1960 records split into 472 categories
Working on watson categories of twitter account: cathcoste
The program found: 223 records split into 116 categories
Working on watson categories of twitter account: tim_cook
The program found: 389 records split into 169 categories
Working on watson categories of twitter account: realDonaldTrump
The program found: 2380 records split into 378 categories
Working on watson categories of twitter account: AstroKatie
The program found: 2085 records split into 515 categories
Working on watson categories of twitter account: EmmanuelMacron
The program found: 3063 records split into 447 categories
Working on watson categories of twitter account: NASA
The program found: 1629 records split into 266 categories
Working on watson categories of twitter account: Jaddo_fr
The program found: 2220 records split into 553 categories
Working on watson 

In [198]:
#show top 20 categories of a specified twitto
CategoriesCollection = mongoHost.categories.paulocoelho
for categories in CategoriesCollection.find({},{'_id' : 0}).sort("count", -1).limit(20):
    print (categories)

{'label': '/art and entertainment/books and literature', 'count': 113}
{'label': '/hobbies and interests/reading', 'count': 61}
{'label': '/business and industrial', 'count': 49}
{'label': '/art and entertainment/movies and tv/movies', 'count': 45}
{'label': '/family and parenting/children', 'count': 40}
{'label': '/science/social science/linguistics/translation', 'count': 39}
{'label': '/society/unrest and war', 'count': 34}
{'label': '/law, govt and politics/espionage and intelligence', 'count': 30}
{'label': '/law, govt and politics/government', 'count': 29}
{'label': '/health and fitness/disorders/mental disorder/panic and anxiety', 'count': 28}
{'label': '/sports/walking', 'count': 28}
{'label': '/religion and spirituality', 'count': 24}
{'label': '/art and entertainment/music', 'count': 23}
{'label': '/art and entertainment/comics and animation/comics', 'count': 23}
{'label': '/law, govt and politics/politics', 'count': 17}
{'label': '/travel/tourist destinations/africa', 'count'

In [206]:
#Collect sentiments from watsonDB and rank them for user

#retrieve tweets sentiments
for watsonCollectionName in mongoHost.watsonDB.collection_names():
    
    print("Working on sentiments of twitter account:", watsonCollectionName)
    #build collection name adress
    watsonCollection = getattr(mongoHost.watsonDB,watsonCollectionName)
    SentimentCollection = getattr(mongoHost.sentiment,watsonCollectionName)
    
    #clean target collection for re-run purposes
    SentimentCollection.delete_many({})
    countAverage = 0
    
    #request emotions on english tweets
    for watsonSentimentAnswer in watsonCollection.find({'language' :  "en"},
                                                      {'emotion' : 1, '_id': 0}):
        #counter used to compute average
        countAverage +=1
        
        #Rebuild dictionary so for loop keep key & value 
        #Naive approach that needs improvement
        watsonSentiments = list(watsonSentimentAnswer["emotion"]["document"]["emotion"])
        watsonSentimentScores = list(watsonSentimentAnswer["emotion"]["document"]["emotion"].values())
        sentimentDict = dict(zip(watsonSentiments,watsonSentimentScores))

        #print (watsonSentiments)
        for sentimentLabel, sentimentValue in sentimentDict.items():
            #print(sentimentLabel, ":", sentimentValue)
            if SentimentCollection.find({'label' : sentimentLabel}).count() > 0:
                #print ("Found !")
                SentimentCollection.update_one({'label': sentimentLabel},{"$inc": {'count': sentimentValue}})
            else:
                #print ("Not found !")
                SentimentCollection.insert_one({'label': sentimentLabel, 'count': sentimentValue})
    
    #average value to get percent
    for sentimentTotal in SentimentCollection.find():
        sentimentAverage = sentimentTotal['count']/countAverage*100
        SentimentCollection.update_one({'label': sentimentTotal['label']},{"$set": {'count': sentimentAverage}})
print ("Done")

Working on sentiments of twitter account: paulocoelho
Working on sentiments of twitter account: cathcoste
Working on sentiments of twitter account: tim_cook
Working on sentiments of twitter account: realDonaldTrump
Working on sentiments of twitter account: AstroKatie
Working on sentiments of twitter account: EmmanuelMacron
Working on sentiments of twitter account: NASA
Working on sentiments of twitter account: Jaddo_fr
Working on sentiments of twitter account: satyanadella
Working on sentiments of twitter account: Jae0Pierre
Working on sentiments of twitter account: EmmaWatson
Working on sentiments of twitter account: BillGates
Working on sentiments of twitter account: BarackObama
Working on sentiments of twitter account: theresa_may
Working on sentiments of twitter account: VasquesXavier
Working on sentiments of twitter account: David_Cameron
Working on sentiments of twitter account: lvanel_IBM
Working on sentiments of twitter account: Bouletcorp
Working on sentiments of twitter accou

In [201]:
#show sentiments of selected user
print ()
for sentimentAveraged in mongoHost.sentiment.EmmanuelMacron.find():
    print (sentimentAveraged)


{'_id': ObjectId('5a552beb2a09f307d4943d45'), 'label': 'sadness', 'count': 17.712670000000003}
{'_id': ObjectId('5a552beb2a09f307d4943d46'), 'label': 'joy', 'count': 43.032970000000006}
{'_id': ObjectId('5a552beb2a09f307d4943d47'), 'label': 'fear', 'count': 11.439422000000002}
{'_id': ObjectId('5a552beb2a09f307d4943d48'), 'label': 'disgust', 'count': 7.913372000000001}
{'_id': ObjectId('5a552beb2a09f307d4943d49'), 'label': 'anger', 'count': 9.811466}
